# Introducción

Se ha notado que hay ciertos profesores que no consiguen acercar las adaptaciones necesarias a sus alumnos. El objetivo de nuestra tecnología sería predecir el nivel académico que va a desempeñar el alumno para estar más pendientes de aquellos alumnos que lo necesiten.

Para ello, se va a utilizar un dataset que contiene información sobre los alumnos y su rendimiento académico. Se va a realizar un análisis exploratorio de los datos para entender mejor la información que contienen y poder realizar un modelo predictivo que nos permita predecir el nivel académico de los alumnos.

# Dataset

## Columnas del dataset que vamos a usar
- `Gender`: Género del alumno
- `Age` : Edad del alumno
- `Study Hour/Day` : Horas de estudio al día
- `Learning mode` : Modalidad de aprendizaje
- `How many hour do you spent daily in social media?` : Horas diarias en redes sociales
- `Average attendance` : Asistencia media
- `With whom you are living?` : Con quién vive
- `What was your previous SGPA?` : Nota media anterior
- `What is your monthly family income?` : Ingresos familiares

Lo primero que hemos hecho ha sido renombrar algunas de las columnas ya que no estaban muy bien redactadas. Además, hemos eliminado las columnas que no vamos a utilizar en nuestro análisis. 

Lo siguiente que hemos hecho ha sido eliminar las filas que contienen valores nulos.

In [5]:
#Limpieza del dataset de los datos de los estudiantes.
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import regex as re

#Cargamos el dataset

data = pd.read_csv('Students_Performance_data_set.csv')

pd.set_option('display.max_columns', None)

data.shape

#Renombramos las columnas que nos interesan

new_columns = {'Gender':'Gender', 'Age':'Age', 'Study Hour/day':'Study time', 'Learning mode':'Learning mode', 'How many hour do you spent daily in social media?':'Time on social media', 'Average attendance on class':'Average attendance', 'With whom you are living with?':'Cohabitants', 'What was your previous SGPA?':'Previous SGPA', 'What is your current CGPA?':'Current CPGA', 'What is your monthly family income?':'Family income'}

data.rename(columns=new_columns, inplace=True)

#Seleccionamos las columnas que nos interesan

columns_of_interest = ['Gender', 'Age', 'Study time', 'Learning mode', 'Time on social media', 'Average attendance', 'Cohabitants', 'Previous SGPA', 'Current CPGA',  'Family income']

data = data[columns_of_interest]

#Eliminamos las filas con valores nulos
data = data.dropna()

## Estandarización de Age
Hemos usado la funcion unique() para ver que tipos de valores pueden tomar estos datos. Hemos apuntados aquellos que no son numéricos o dan un rango y hemos decidido cambiarlos a numéricos. Esto se puede ver el bloque de código siguiente.

In [6]:
data['Age'].unique()

# notamos que hay 6 valores que se salen de un numero como tal asi que usamos un replace para cambiarlos a un valor que no afecte el analisis
data['Age'] = data['Age'].replace({
    '21+' : 21,
    '23.6' : 23,
    '20+' : 20,
    '22+' : 22,
    '24+' : 24,
    '20 years 6 months' : 20
})

## Estandarizacion de tiempos

Lo que hemos hecho en el codigo que sigue este bloque de markdown es estandarizar los tiempos para poder tener los datos correctos tanto como para el Study Time como para el Time on Social Media.

In [7]:
### Usamos la función unique para ver los valores únicos de la columna y asi saber como estandarizarlos
# print(data['Study time'].unique())

def standarise_time(time): # sigue teniendo un error en la función cuando tiene un valor de minutos y otro seguido de horas
    # we extract the number and hour/minute from the string
    hours_numeric = re.findall(r'\d+', str(time))
    
    # extract the numeric value for the hours
    if 'hours' in str(time).lower() or 'hour' in str(time).lower() or 'hrs' in str(time).lower() or 'hr' in str(time).lower():
        if hours_numeric:
            return int(hours_numeric[0])
        else:
            return None
    # extract the numeric value for the minutes and convert them to hours convert to lower case to avoid case sensitivity
    elif 'minutes' in str(time).lower() or 'minute' in str(time).lower() or 'mins' in str(time).lower() or 'min' in str(time).lower() :
        if hours_numeric:
            return int(hours_numeric[0])/60
        else:
            return None

    # Some of the data is written in a range format, so we need to calculate the average
    elif '-' in str(time) or 'to' in str(time).lower() or 'or' in str(time).lower() or '/' in str(time):
        # we calculate the average
        if len(hours_numeric) == 2:
            return (int(hours_numeric[0]) + int(hours_numeric[1]))/2
        else:
            return None
    # if the data is in a numeric format, we just return the number
    elif hours_numeric:
        return int(hours_numeric[0])
    else:
        return None

data['Study stand'] = data['Study time'].apply(standarise_time)
## TODO si queremos inferir aquellos datos que no se pudieron estandarizar en vez de hacer la media guarra
study_mean = data['Study stand'].mean()
data['Study stand'] = data['Study stand'].fillna(study_mean)


data['Social stand'] = data['Time on social media'].apply(standarise_time)
## TODO si queremos inferir aquellos datos que no se pudieron estandarizar en vez de hacer la media guarra
social_mean = data['Social stand'].mean()
data['Social stand'] = data['Social stand'].fillna(social_mean)


## Estandarizacion de Attendance

En el siguiente bloque de codigo hemos implementado la funcion standarise_attendance(value) convirtiendo los datos para que siguan el mismo formato compuesto por un numero y ya esta. Para aquellos valores que sean NaN, cogeremos el valor medio de la columna. 

In [8]:
def standarise_attendance(attendance):
    
    # primero quitamos todos los caracteres que no sean alfanuméricos o un signo de porcentaje
    value = re.sub(r'[^a-zA-Z0-9%]', '', str(attendance))
    
    if '%' in value:
        value = re.search(r'\d+', value).group()
        value = int(value)
    # si no es un porcentaje intentar convertirlo a un número
    else:
        try:
            value = int(value)
        except ValueError:
            value = float('nan')
            
    # asegurarse de que el valor esté entre 0 y 100
    value = min(max(value, 0), 100)
    return value

data['Attendance stand'] = data['Average attendance'].apply(standarise_attendance)

## TODO si queremos inferir aquellos datos que no se pudieron estandarizar en vez de hacer la media guarra
attendance_mean = data['Attendance stand'].mean()
data['Attendance stand'] = data['Attendance stand'].fillna(attendance_mean)

## Estandarizacion de CGPA y SGPA

En este caso en vez de definir una funcion como todos los datos que estan en formato numerico son correctos usamos to_numeric() de pandas, cuando se encuentre con una valor que no se puede convertir a numerico devolvemos NaN con errors='coerce'. Para aquellos datos que sean NaN, hacer la media de los datos que si que se pueden convertir a numerico y rellenar los NaN con esa media.

In [9]:
data["Current CPGA stand"] = pd.to_numeric(data["Current CPGA"], errors='coerce')
## TODO si queremos inferir aquellos datos que no se pudieron estandarizar en vez de hacer la media guarra
cpga_mean = data["Current CPGA stand"].mean()
data["Current CPGA stand"].fillna(cpga_mean, inplace=True)


data["Previous SGPA stand"] = pd.to_numeric(data["Previous SGPA"], errors='coerce')
## TODO si queremos inferir aquellos datos que no se pudieron estandarizar en vez de hacer la media guarra
sgpa_mean = data["Previous SGPA stand"].mean()
data["Previous SGPA stand"].fillna(sgpa_mean, inplace=True)

/tmp/ipykernel_15880/834309524.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Current CPGA stand"].fillna(cpga_mean, inplace=True)
/tmp/ipykernel_15880/834309524.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=T

## Estandandarizacion de Family Income
Como siempre usamos data.unique() para ver los valores que toman los datos y de ahi decidir como vamos a tratar los datos que consideramos "anomalos" o mal escritos.

1. Convertir de otras monedas y otras magnitudes a bdt 10E
2. Transformar los rangos a un valor numerico
3. Quitar el BDT que es la moneda de Bangladesh
4. Dejar numeros limpios. Alguna gente a puesto valores como "Approximately 30000". Lo hemos convertido al valor numerico
5. Cambiar los valores que no son numericos a la media de income del dataset

In [10]:

# transform currency to bdt 10E
def convert_to_bdt_standard(income):
    if isinstance(income, str) and '$' in income.lower():
        number = re.findall(r'\d+', income)
        if number:
            return int(number[0])*109.7 #Conversion de dolar a taka 10.04.2024
    if isinstance(income, str) and 'lac' in income.lower():
        number = re.findall(r'\d+', income)
        if number:
            return int(number[0])*100000 #Conversion de lakh(lac) a estandard
    if isinstance(income, str) and 'k' in income.lower():
        number = re.findall(r'\d+', income)
        if number:
            return int(number[0])*1000
    return income

# transformamos rangos a la media
def remove_range(income):
    if isinstance(income, str) and ('-' in income or '/' in income) :
        numbers = re.findall(r'\d+', income)
        if len(numbers) == 2:
            return (int(numbers[0]) + int(numbers[1]))/2
    return income
    
# quitando el BDT
def remove_BDT(income):
    if isinstance(income, str) and ('bdt' in income.lower() or 'DBT' in income or 'BTD' in income or 'taka' in income):
        number = re.findall(r'\d+', income)
        if number:
            return int(number[0])
    return income

def clean_numbers(income):
    if isinstance(income, str):
        number = re.findall(r'\d+', income)
        if number:
            return number[0]
    return income

# apply the function to the column
data['Family income stand'] = data['Family income'].apply(convert_to_bdt_standard)
data['Family income stand'] = data['Family income stand'].apply(remove_range)
data['Family income stand'] = data['Family income stand'].apply(remove_BDT)
data['Family income stand'] = data['Family income stand'].apply(clean_numbers)
data['Family income stand'] = pd.to_numeric(data['Family income stand'], errors='coerce')

## TODO si queremos inferir aquellos datos que no se pudieron estandarizar en vez de hacer la media guarra
mean_income = data['Family income stand'].mean()
data['Family income stand'] = data['Family income stand'].fillna(mean_income)
data['Family income stand'] = data['Family income stand'].astype(int) 


## Conversion de datos

Convertimos los numeros a numericos

In [11]:
data['Family income stand'] = pd.to_numeric(data['Family income stand'], errors='coerce')
data['Attendance stand'] = pd.to_numeric(data['Attendance stand'], errors='coerce')
data['Study stand'] = pd.to_numeric(data['Study stand'], errors='coerce')
data['Social stand'] = pd.to_numeric(data['Social stand'], errors='coerce')
data['Age stand'] = pd.to_numeric(data['Age'], errors='coerce')

## creamos un nuevo dataset con las columnas que nos interesan
new_columns = ['Gender', 'Age stand', 'Study stand', 'Learning mode', 'Social stand', 'Attendance stand', 'Cohabitants', 'Previous SGPA stand', 'Current CPGA stand', 'Family income stand']

data = data[new_columns]

## data.dropna()
display(data)

## mostrar numero de valores nulos
print(data.isnull().sum())

data.to_csv('Students_Performance_data_set_cleaned.csv', index=False)
dataCL = pd.read_csv('Students_Performance_data_set_cleaned.csv')

,Gender,Age stand,Study stand,Learning mode,Social stand,Attendance stand,Cohabitants,Previous SGPA stand,Current CPGA stand,Family income stand
7,Female,22,2.0,Offline,2.0,100.000000,Bachelor,3.80,3.64,32500
11,Male,22,3.0,Offline,2.0,90.000000,Bachelor,3.40,3.53,20000
15,Male,20,2.0,Offline,1.0,95.000000,Family,3.93,3.89,30000
18,Male,21,1.0,Online,3.0,88.333006,Family,3.10,3.50,207414
20,Female,21,3.0,Offline,2.0,96.000000,Bachelor,3.81,3.65,30000
...,...,...,...,...,...,...,...,...,...,...
1189,Female,20,1.0,Online,2.0,46.000000,Bachelor,2.65,3.77,180000
1190,Male,23,4.0,Offline,4.0,100.000000,Bachelor,2.50,2.22,200000
1191,Male,22,3.0,Offline,2.0,100.000000,Family,1.56,2.78,200000
1192,Female,25,5.0,Offline,3.0,100.000000,Bachelor,1.40,2.52,210000


Gender                 0
Age stand              0
Study stand            0
Learning mode          0
Social stand           0
Attendance stand       0
Cohabitants            0
Previous SGPA stand    0
Current CPGA stand     0
Family income stand    0
dtype: int64


## Separación datos para entrenamieto

In [18]:
data_columns = ['Age stand', 'Study stand', 'Social stand', 'Attendance stand', 'Previous SGPA stand', 'Family income stand']
data_target = ['Current CPGA stand']

dataCL.data = dataCL[data_columns]
dataCL.target = dataCL[data_target]

X_train = dataCL.data[:-20]
y_train = dataCL.target[:-20]
X_test = dataCL.data[-20:]
y_test = dataCL.target[-20:]

## KNN

In [20]:
from sklearn.neighbors import KNeighborsRegressor

# Crear el modelo
knn = KNeighborsRegressor(n_neighbors=3)

# Ajustar el modelo a los datos de entrenamiento
knn.fit(X_train, y_train)

prediction = knn.predict(X_test)
print("Prediccion")
print(prediction)
print("Test")
print(y_test)

knn.score(X_test, y_test)

Prediccion
[[3.5       ]
 [3.43      ]
 [3.70333333]
 [3.5       ]
 [3.        ]
 [3.        ]
 [3.45666667]
 [3.47666667]
 [3.35666667]
 [3.35666667]
 [3.71666667]
 [2.72333333]
 [2.6       ]
 [2.99333333]
 [3.17666667]
 [3.16666667]
 [3.17666667]
 [3.17666667]
 [3.24      ]
 [3.16      ]]
Test
      Current CPGA stand
1020                3.12
1021                3.60
1022                2.80
1023                3.77
1024                2.93
1025                2.94
1026                2.78
1027                2.54
1028                2.83
1029                2.85
1030                3.66
1031                3.88
1032                3.90
1033                3.44
1034                3.43
1035                3.77
1036                2.22
1037                2.78
1038                2.52
1039                2.88


-0.6971760829166422

## Árboles de decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)

decision_tree = decision_tree.fit(X_train, y_train)
r = export_text(decision_tree, feature_names=data.feature_names)
print(r)
decision_tree.score(X_test, y_test)

## SVM

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear', random_state=0)

classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)
print(y_pred)
model.score(X_test, y_test) ## no se lo que es model

## Red Neuronal

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=100, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300, verbose=True).fit(X_train, y_train)
clf.predict_proba(X_test[:1])
y_pred = clf.predict(X_test[:, :])
print(y_test)
print(y_pred)
clf.score(X_test, y_test)